In [4]:
import cv2
import numpy as np
import time
import mediapipe as mp
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume
def findHandMarker(frame,hands):
    '''This function findsd the position of tip of thumb and index finger
    and marks it with a circle'''
    point=[]
    framergb=cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    result=hands.process(framergb)
    if result.multi_hand_landmarks:
        use_hand=result.multi_hand_landmarks
        
        for id,lm in enumerate(use_hand[0].landmark):
            h,w,c=frame.shape
            if id==4:
                x1,y1=int(w*lm.x),int(h*lm.y)
                point.append([x1,y1])
                cv2.circle(frame,(x1,y1),10,(255,0,255),-1)
            elif id==8:
                x2,y2=int(w*lm.x),int(h*lm.y)
                point.append([x2,y2])
                cv2.circle(frame,(x2,y2),10,(255,0,255),-1)
            
            
            
    return frame,point

def setVol(length):
    '''This function sets the volume of system according to the distance between tip of thumb and index finger'''
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(
        IAudioEndpointVolume._iid_, CLSCTX_ALL, None)
    volume = cast(interface, POINTER(IAudioEndpointVolume))
    #volume.GetMute()
    #volume.GetMasterVolumeLevel()
    sysVol=volume.GetVolumeRange() 
    minVol=sysVol[0]
    maxVol=sysVol[1]
    vol=np.interp(length,[25,180],[minVol,maxVol])
    volume.SetMasterVolumeLevel(vol, None)

#main body of code
cap=cv2.VideoCapture(0)
ctime=0
ptime=0

mpDraw=mp.solutions.drawing_utils
mphands=mp.solutions.hands
hands=mphands.Hands()
while cap.isOpened() :
    barLength=80
    ret,frame=cap.read()
    frame,point=findHandMarker(frame,hands)
    if len(point)!=0:
        cv2.line(frame,(point[0][0],point[0][1]),(point[1][0],point[1][1]),(255,0,255),5)
        length=math.hypot(point[1][0]-point[0][0],point[1][1]-point[0][1])
        #print(length)# helps to get the min and max length
        midx,midy=(point[1][0]+point[0][0])//2,(point[1][1]+point[0][1])//2
        cv2.circle(frame,(midx,midy),10,(255,0,255),-1)
        setVol(length)
        cv2.rectangle(frame,(10,30),(30,80),(255,255,0),2)
        barLength=np.interp(length,[25,180],[80,30])
        cv2.rectangle(frame,(10,int(barLength)),(30,80),(255,255,0),-1)
    ctime=time.time()
    fps=1/(ctime-ptime)
    cv2.putText(frame,str(int(fps)),(5,30),cv2.FONT_HERSHEY_SIMPLEX,1,(255,0,255),2)
    ptime=ctime
    cv2.imshow('frame',frame)
    if cv2.waitKey(1) & 0xFF==27:
        break
cap.release()
cv2.destroyAllWindows()
    
               
        
        
